In [ ]:
# different statistics and descriptive plots regarding irt2 metrics

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import irt2
from irt2.dataset import IRT2

def load_datasets():
    root = irt2.ENV.DIR.DATA / 'irt2' / 'cde'
    return {
        path.name: IRT2.from_dir(path=path) for path in (
            root / 'tiny',
            root / 'small',
            root / 'medium',
            root / 'large',
        )
    }

datasets = load_datasets()
for name, dataset in datasets.items():
    print(f"\nloaded {name}:")
    print(f"{dataset.graph}")
    print(f"got ratios for {len(dataset.relations)} relations")



loaded tiny:
IRT graph: [IRT2/CDE-XS] (12389 entities)
got ratios for 5 relations

loaded small:
IRT graph: [IRT2/CDE-S] (14207 entities)
got ratios for 13 relations

loaded medium:


IRT graph: [IRT2/CDE-M] (15020 entities)
got ratios for 46 relations

loaded large:


IRT graph: [IRT2/CDE-L] (15020 entities)
got ratios for 46 relations


In [5]:
# load source graph

import irt2
from irt2.graph import Graph
from irt2.graph import load_graph


def load_source_graph(config):
    return load_graph(
        config['graph loader'],
        config['graph name'],
        *[irt2.ENV.DIR.ROOT / path for path in config['graph loader args']],
        **{k: irt2.ENV.DIR.ROOT / path for k, path in config['graph loader kwargs'].items()},
    )


source_graph = load_source_graph(config=datasets['large'].config['create'])
print(str(source_graph))

IRT graph: [CodEx-M] (17050 entities)


In [23]:
# We want to check whether the greedy approach to subsampling actually yields
# comparable ratios between relations or if some larger bias can be spotted.
#
# (Think about having a relation "fifty" where 50% of all vertices take part and
# another "hunny" where all vertices are included. In the most extreme case, if
# by chance the greedy approach saturates "hunny" without ever encountering any
# "fifty" vertices, no vertex for "fifty" will be included in the closed world)

from irt2.graph import Relation
import matplotlib.pyplot as plt


# len(datasets['tiny'].ratios[0].heads)

def plot_ratios(relations):

    # prepare data

    


    # plot

    n = len(datasets)

    fig, axes = plt.subplots(1, n)
    fig.set_figwidth(n * 4)

    axes[0].set_title('foo')

    splits = 'a', 'b', 'c', 'd'
    heads = 10, 20, 30, 40
    tails = 3, 2, 5, 100

    axes[0].bar(x=splits, height=heads)
    axes[0].bar(x=splits, height=tails, bottom=heads)


# relations = {'original': Relation.from_graph(g=source_graph)}
relations |= {name: dataset.ratios for name, dataset in datasets.items()}

plot_ratios(relations=relations)



AssertionError: P50:author (rid=3) has no triples assigned